In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

FHIR_DIR = "/content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)"
print("FHIR bundles will be read from:", FHIR_DIR)

Mounted at /content/drive
FHIR bundles will be read from: /content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)


In [2]:
%%bash
# (re‑)create venv
python -m pip install --quiet virtualenv
python -m virtualenv /content/fhir_env

# activate & upgrade pip
source /content/fhir_env/bin/activate
pip install --quiet --upgrade pip

# core libs (NumPy 2.0, PyTorch 2.3 CUDA 12, etc.)
pip install --quiet numpy==2.0.0 \
    --extra-index-url https://download.pytorch.org/whl/cu121 \
    torch==2.3.0 torchvision==0.18.0

# helper libs for the clinical assistant
pip install --quiet \
    fhir.resources==7.1.0 \
    sentence-transformers==2.7.0 \
    transformers==4.43.2 accelerate==0.30.1 bitsandbytes==0.43.0 \
    langchain==0.2.0 langchain-community==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 34.7 MB/s eta 0:00:00
created virtual environment CPython3.11.13.final.0-64 in 329ms
  creator CPython3Posix(dest=/content/fhir_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 148.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 185.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 169.9 MB/s et

In [ ]:
%%bash
source /content/fhir_env/bin/activate

# build prerequisites
sudo apt-get update -qq
sudo apt-get install -y --no-install-recommends \
     swig python3-dev build-essential cmake git libopenblas-dev libomp-dev

# clone & compile Faiss 1.7.4 with CUDA
rm -rf faiss
git clone --branch v1.7.4 https://github.com/facebookresearch/faiss.git
cd faiss
cmake -B build -DFAISS_ENABLE_GPU=ON -DFAISS_ENABLE_PYTHON=ON \
               -DFAISS_ENABLE_C_API=OFF -DCMAKE_BUILD_TYPE=Release \
               -DPython_EXECUTABLE=$(which python)
cmake --build build --target faiss faiss_avx2 swigfaiss -j $(nproc)

# install python bindings from the build tree
cd build/faiss/python
pip install .   # produces an *empty* faiss package + _swigfaiss.so

In [ ]:
import site, sys, pathlib
VENV = "/content/fhir_env"
sp   = next(pathlib.Path(VENV).rglob("site-packages"))
if str(sp) not in sys.path:
    sys.path.insert(0, str(sp))
print("site‑packages added:", sp)

In [ ]:
import sys, importlib.util, types, pathlib

# locate the raw SWIG extension
spec = importlib.util.find_spec("_swigfaiss")
if spec is None:
    candidate = "/content/faiss/build/faiss/python"
    if candidate not in sys.path:
        sys.path.insert(0, candidate)
    spec = importlib.util.find_spec("_swigfaiss")
if spec is None:
    raise ImportError("Cannot find _swigfaiss.so – did the build succeed?")

# load extension and create façade
_swf  = importlib.import_module("_swigfaiss")
faiss = types.ModuleType("faiss")
for name in dir(_swf):
    if not name.startswith("__"):
        setattr(faiss, name, getattr(_swf, name))
sys.modules["faiss"] = faiss
print("Patched faiss facade with", len(dir(faiss)), "symbols")

In [ ]:
import faiss, torch, numpy as np
d = 32
cpu_index = faiss.IndexFlatL2(d)
res       = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

xb = np.random.randn(10_000, d).astype('float32')
gpu_index.add(xb)
D, _ = gpu_index.search(xb[:5], 3)
print("✓ GPU Faiss operational – distances shape:", D.shape)

print("CUDA visible to Torch?:", torch.cuda.is_available(),
      "| device name:", torch.cuda.get_device_name(0))

In [ ]:
#NEW RUN above, ignore everything below!!!

In [1]:
%%bash
# 0️⃣ Install virtualenv (only ~6 s)
python -m pip install --quiet virtualenv

# 1️⃣ Create the env — virtualenv bundles its own pip+setuptools, so no ensurepip needed
python -m virtualenv /content/fhir_env

# 2️⃣ Activate it for this *shell* and upgrade pip
source /content/fhir_env/bin/activate
pip install --quiet --upgrade pip

echo "✅ Virtualenv ready at /content/fhir_env"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 33.0 MB/s eta 0:00:00
created virtual environment CPython3.11.13.final.0-64 in 327ms
  creator CPython3Posix(dest=/content/fhir_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
✅ Virtualenv ready at /content/fhir_env


In [2]:
%%bash
source /content/fhir_env/bin/activate

# 1️⃣ Core numeric stack
pip install --quiet numpy==2.0.0 \
                       --extra-index-url https://download.pytorch.org/whl/cu121 \
                       torch==2.3.0 torchvision==0.18.0

# 2️⃣ Clinical‑assistant libraries
pip install --quiet faiss-cpu==1.8.0 \
                   fhir.resources==7.1.0 \
                   sentence-transformers==2.7.0 \
                   transformers==4.43.2 \
                   accelerate==0.30.1 \
                   bitsandbytes==0.43.0 \
                   langchain==0.2.0 langchain-community==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 127.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 217.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 172.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 120.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 178.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 131.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 149.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%bash
source /content/fhir_env/bin/activate

# ── 1.  Build prerequisites ────────────────────────────────────────────────
sudo apt-get update -qq
sudo apt-get install -y swig python3-dev
sudo apt-get install -y --no-install-recommends \
     build-essential cmake git libopenblas-dev libomp-dev

# ── 2.  Fresh clone of Faiss v1.7.4 ────────────────────────────────────────
rm -rf faiss
git clone --branch v1.7.4 https://github.com/facebookresearch/faiss.git
cd faiss

# ── 3.  Configure + compile (GPU build, Python bindings) ───────────────────
cmake -B build -DFAISS_ENABLE_GPU=ON \
               -DFAISS_ENABLE_PYTHON=ON \
               -DFAISS_ENABLE_C_API=OFF \
               -DCMAKE_BUILD_TYPE=Release \
               -DPython_EXECUTABLE=$(which python)
cmake --build build --target faiss faiss_avx2 swigfaiss -j $(nproc)

# ── 4.  Install the bindings directly from the build tree ──────────────────
cd build/faiss/python
pip install .

In [ ]:
import site, sys, pathlib
VENV = "/content/fhir_env"
site_packages = next(pathlib.Path(VENV).rglob("site-packages"))
if str(site_packages) not in sys.path:
    sys.path.insert(0, str(site_packages))

In [ ]:
import sys, importlib.util, types, pathlib

# --- locate the compiled _swigfaiss extension ---------------------------
spec = importlib.util.find_spec("_swigfaiss")
if spec is None:
    candidate = "/content/faiss/build/faiss/python"
    if candidate not in sys.path:
        sys.path.insert(0, candidate)
    spec = importlib.util.find_spec("_swigfaiss")
if spec is None:
    raise ImportError("Could not locate _swigfaiss.so; did you build Faiss?")

# --- import it and create a façade called `faiss` -----------------------
_swf  = importlib.import_module("_swigfaiss")
faiss = types.ModuleType("faiss")
for name in dir(_swf):
    if not name.startswith("__"):
        setattr(faiss, name, getattr(_swf, name))
sys.modules["faiss"] = faiss      # so `import faiss` works elsewhere

# --- smoke test ---------------------------------------------------------
import numpy as np, torch
d = 32
cpu_index = faiss.IndexFlatL2(d)
gpu_res   = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(gpu_res, 0, cpu_index)
xb = np.random.randn(10_000, d).astype("float32")
gpu_index.add(xb)
D, _ = gpu_index.search(xb[:5], 3)
print("✓ GPU Faiss working – result shape:", D.shape)

In [7]:
import faiss, torch, pathlib, numpy

faiss_version = getattr(faiss, "__version__", "local‑build")   # graceful fallback
print("FAISS :", faiss_version, "| GPUs:", faiss.get_num_gpus())
print("Path  :", pathlib.Path(faiss.__file__).parent)
print("NumPy :", numpy.__version__)

AttributeError: module 'faiss' has no attribute 'get_num_gpus'

In [8]:
import faiss, torch, numpy as np

print("PyTorch sees CUDA:", torch.cuda.is_available(), "GPUs:", torch.cuda.device_count())

# Create a small index on the CPU…
d = 64
cpu_index = faiss.IndexFlatL2(d)   # exact L2 search

# Move it to GPU 0
# StandardGpuResources always exists when FAISS_ENABLE_GPU=ON
gpu_res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(gpu_res, 0, cpu_index)

# Add some random vectors and run a query
xb = np.random.random((1000, d)).astype('float32')
xq = np.random.random((5, d)).astype('float32')

gpu_index.add(xb)
D, I = gpu_index.search(xq, k=3)

print("GPU search results shape:", D.shape)  # should be (5, 3)
print("First row distances:", D[0])

PyTorch sees CUDA: True GPUs: 1


AttributeError: module 'faiss' has no attribute 'IndexFlatL2'

In [9]:
import faiss
print(dir(faiss))

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [ ]:
import site, sys, pathlib, importlib
VENV = "/content/fhir_env"
site_packages = next(pathlib.Path(VENV).rglob("site-packages"))
if str(site_packages) not in sys.path:
    sys.path.insert(0, str(site_packages))

# quick sanity‑check
import numpy, torch, faiss, transformers
print("NumPy :", numpy.__version__)
print("Torch :", torch.__version__)
print("FAISS :", faiss.__version__)
print("tfms  :", transformers.__version__)

In [ ]:
##############################################################################
# 0.  SCRUB THE OLD ENVIRONMENT COMPLETELY
##############################################################################
!pip uninstall -y -q numpy opencv-python-headless thinc numba tensorflow faiss-cpu || true

##############################################################################
# 1.  INSTALL A SINGLE, COMPATIBLE NUMPY WHEEL
#     NumPy 2.0.x is               >=2  and <2.1   → ok for:
#       • opencv‑python‑headless (>=2,<3)
#       • thinc (>=2,<3)
#       • tensorflow 2.18 (>=1.26,<2.1)
#       • numba 0.60      (>=1.22,<2.1)
##############################################################################
!pip install --quiet --no-cache-dir "numpy==2.0.0"

##############################################################################
# 2A. OPTIONAL: KEEP TensorFlow & Numba (if you really need them later)
# Comment‑out the next two lines if you don’t need either package.
##############################################################################
!pip install --quiet --no-cache-dir numba==0.60.0 tensorflow==2.18.0

##############################################################################
# 2B.  INSTALL / REINSTALL THE TOOLS NEEDED FOR THE FHIR ASSISTANT
##############################################################################
!pip install --quiet --no-cache-dir \
    opencv-python-headless==4.12.0.88 \
    thinc==8.3.6 \
    fhir.resources==7.1.0 \
    sentence-transformers==2.7.0 \
    transformers==4.43.2 \
    accelerate==0.30.1 \
    bitsandbytes==0.43.0 \
    langchain==0.2.0 \
    langchain-community==0.2.0

##############################################################################
# 3.  (RE)INSTALL FAISS **FROM SOURCE** SO IT LINKS AGAINST NUMPY 2
#     Compiling takes ~4 minutes on an A100 but guarantees ABI compatibility.
##############################################################################
!pip install --quiet --no-binary faiss-cpu --no-cache-dir faiss-cpu==1.8.0

##############################################################################
# 4.  VERIFY THERE IS EXACTLY **ONE** NUMPY IN THE ENVIRONMENT
##############################################################################
import subprocess, sys, importlib
for m in ("numpy","cv2","thinc","faiss"):
    mod = importlib.import_module(m)
    print(f"{m:5s}", mod.__version__)
subprocess.run([sys.executable,"-m","pip","check"], check=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 146.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 311.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.48.0 requires numba>=0.54, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
pynndescent 0.5.13 requires numba>=0.51.2, which is not installed.
umap-learn 0.5.9.post2 requires numba>=0.51.2, which is not installed.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, which is not installed.
stumpy 1.13.0 requires numba>=0.57.1, which is not installed.
librosa 0.11.0 requires numba>=0.51.0, which is not installed.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, which is not installed.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
dask-cuda 25.2.0 requires numba<0.61.0a

AttributeError: module 'cv2' has no attribute '__version__'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json, glob, os, sqlite3, uuid, datetime as dt, re, textwrap
from pathlib import Path
from typing import List, Dict, Any

DATA_DIR   = "/content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)"        # point this to **all** your .json bundles
SQLITE_DB  = "/content/structured_fhir.db"  # lightweight relational store
INDEX_DIR  = "/content/faiss_index"         # vector store on disk
EMB_MODEL  = "thenlper/gte-large"           # 430 MB, 1024‑dim embeddings, MIT licence
LLM_MODEL  = "mistralai/Mistral-7B-Instruct-v0.2"  # Apache‑2.0, fits in 16 GB 4‑bit